# Evaluate Metdata columns

In [1]:
import os
import pandas as pd
from unipark import Preprocessor, CodeBookParser
from unipark import MetadataManipulator as MdManipulator
from unipark.utils.frame import get_finishers, get_pausers, get_nonstarters
import matplotlib.pyplot as plt
import json

c:\Users\juf\Workspace\BTH\UniPark_Parsing\unipark\utils\frame.py:35: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  def translate_multiple_choice(df, translator_map, prefix, int_converter = lambda x: x is not 0):


In [15]:
# load the paths to the input CSV, codebook, and figures save directory from the paths.json file located in the same directory
with open('paths.json') as f:
    paths = json.load(f)
    input_csv = paths['input_csv']
    codebook_path = paths['codebook_path']
    figures_save_dir = paths['figures_save_dir_metadata']

In [16]:
# ensure the existence of the directory where the figures are to be saved
if not os.path.exists(figures_save_dir):
    os.makedirs(figures_save_dir)

In [17]:
cbp = CodeBookParser(codebook_path)

codebook = cbp.get_codebook()
page_name_by_id = {}
for page in codebook['pages']:
    page_name_by_id[page['id']] = page['title']
    
def to_named_page(x):
    return page_name_by_id[str(x)] if str(x) in page_name_by_id else str(x)

KeyError: 143

In [ ]:
pproc = Preprocessor(pd.read_csv(input_csv,sep=";"))
pproc.apply_manipulator(MdManipulator())
pproc.drop_removable()

In [ ]:
data = pproc.data
md = pproc.get_metadata()

## Participant count

In [ ]:
print("Total participants:\t{}".format(len(md)))
print("\tfinished:\t{}".format(len(get_finishers(md))))
print("\tpaused:\t\t{}".format(len(get_pausers(md))))
print("\tunstarted:\t{}".format(len(get_nonstarters(md))))

In [ ]:
dispcodes = data["dispcode_named"].value_counts().sort_index()
print(dispcodes)
dispcodes.plot.pie()

## Pages paticipants stopped on

In [ ]:
ax = md["lastpage"].value_counts().rename(index=to_named_page).sort_index(ascending=False).plot.pie()
ax.set_title('Last page of participants')
plt.savefig(os.path.join(figures_save_dir,'last_page_pie.png'), dpi=1200, bbox_inches='tight')

### Those who paused stopped on page...

In [ ]:
paused_on_page = get_pausers(md)["lastpage"].value_counts().rename(index=to_named_page).sort_index()
print("Number of participants that paused: " + paused_on_page)
if len(paused_on_page) > 0:
    ax = paused_on_page.plot.bar(legend=True)
    ax.set_title('#Participants who stopped on page')
    plt.savefig(os.path.join(figures_save_dir,'page_stopped_bar.png'), dpi=1200, bbox_inches='tight')

## Platform distribution

In [ ]:
pf = data['platform']
pff = get_finishers(data)['platform']
vcs = pd.DataFrame.from_dict({'all':pf.value_counts(), 'finishers':pff.value_counts()})
vcs_rel = pd.DataFrame.from_dict({'all':pf.value_counts()/len(pf), 'fins':pff.value_counts()/len(pff)})
ax = pf.value_counts().plot.pie()
ax.set_title('OS usage of participants')
plt.savefig(os.path.join(figures_save_dir,'os_distribution_pie.png'), dpi=1200, bbox_inches='tight')
plt.show()
ax = pff.value_counts().plot.pie()
ax.set_title('OS usage of finishers')

In [ ]:
vcs = pd.DataFrame.from_dict({'all':pf.value_counts(), 'finishers':pff.value_counts()})
vcs_rel = pd.DataFrame.from_dict({'all':pf.value_counts()/len(pf), 'finishers':pff.value_counts()/len(pff)})
ax = vcs_rel.plot.bar()
ax.set_title("Relative OS usage of participants")
plt.savefig(os.path.join(figures_save_dir,'os_distribution_bar.png'), dpi=1200, bbox_inches='tight')

In [ ]:
data['platform'].value_counts()/len(data)